created 4-9-24

Notebook is a modification of `initialize_drifters.py` from the transition matrix project found [here](https://github.com/cspencerjones/transition_matrix_project/blob/main/double_gyre/initialize_drifters.py).

This notebook is used to release drifters where desired using a `drifters.res.nc` file. In this case, one drifter is released at the center of every depth-longitude grid cell at 10$^{\circ}$S. Longitudes are evenly spaced. Depths are spaced according to a 32-level ALE coordinate grid created in [this INPUT notebook](../t_m_p_wind/6-21_build_INPUT/vgrid_32_layers_build.ipynb).


In [1]:
import xarray as xr
import netCDF4 as nc
import numpy as np

In [2]:
def createParticlesNC(fnam,ids,ines,jnes,start_lats,start_lons,lons,lats,levs,start_ds,days):
    f=nc.Dataset(fnam,'w',format='NETCDF3_64BIT_OFFSET',clobber=True)
    idim=f.createDimension('i',None)
    ivv=f.createVariable('i',np.float64)
    iv=f.createVariable('drifter_num',np.int32,('i',))
    inev=f.createVariable('ine',np.int32,('i',))
    jnev=f.createVariable('jne',np.int32,('i',))
    start_latv = f.createVariable('start_lat',np.float64,('i',))
    start_lonv = f.createVariable('start_lon',np.float64,('i',))
    start_dv = f.createVariable('start_d',np.float64,('i',))
    latv=f.createVariable('lat',np.float64,('i',))
    lonv=f.createVariable('lon',np.float64,('i',))
    levv=f.createVariable('depth',np.float64,('i',))
    dv=f.createVariable('time',np.float64,('i',))

    f.file_format_major_version=1
    f.file_format_minor_version=1
    f.time_axis = 0
    iv.long_name='identification of the drifter'
    iv.units='dimensionless'
    iv.packing=0
    inev.long_name='i index'
    inev.units='none'
    inev.packing=0
    jnev.long_name='j index'
    jnev.units='none'
    jnev.packing=0
    start_latv.long_name='start latitude'
    start_latv.units='degrees_N'
    start_lonv.long_name='start_longitude'
    start_lonv.units='degrees_E'
    start_dv.long_name='start_depth'
    start_dv.units='start depth below surface'
    lonv.long_name='longitude'
    lonv.units='degrees_E'
    latv.long_name='latitude'
    latv.units='degrees_N'
    levv.long_name= 'depth below surface'
    levv.units='m'
    dv.units='days since 1900-01-01 00:00:00'
    ivv[:]=len(ids[:])
    iv[:] = ids[:]
    inev[:]=ines[:]
    jnev[:]=jnes[:]
    start_latv = start_lats[:]
    start_lonv = start_lons[:]
    start_dv = start_ds[:]
    lonv[:]=lons[:]
    latv[:]=lats[:]
    levv[:]=levs[:]
    dv[:]=days[:]
    f.sync()
    f.close()

start_lat1D = np.arange(-10.5,-9.5,1)
start_lon1D = np.arange(0.5,40.5,1) 
lon1D = np.arange(0.5,40.5,1)
lat1D = np.arange(-10.5,-9.5,1)

start_lonmesh,start_latmesh = np.meshgrid(start_lon1D,start_lat1D)
lonmesh,latmesh = np.meshgrid(lon1D,lat1D)

sizemesh = lonmesh.ravel().size
nolevels = 32
noparts = sizemesh*nolevels

zllevels = np.array([   6.5,   20.5,   36.5,   55. ,   76.5,  101. ,  129. ,  161. ,  197.5,
        239.5,  287.5,  342. ,  403.5,  473. ,  551.5,  639.5,  738.5,  849.5,
        973. , 1110. , 1261. , 1426.5, 1607. , 1803. , 2014.5, 2240.5, 2480.5,
       2733.5, 2997.5, 3270.5, 3550. , 3845.5])
    
id_list = np.arange(1,noparts+1,1).tolist()
ines_list = [0] * noparts
jnes_list = [0] * noparts
start_lats_list = np.repeat(start_latmesh.ravel().tolist(),nolevels)
start_lons_list = np.repeat(start_lonmesh.ravel().tolist(),nolevels)
start_ds_list = np.repeat(np.expand_dims(zllevels,0),sizemesh,axis=0).ravel()
lons_list = np.repeat(lonmesh.ravel().tolist(),nolevels)
lats_list = np.repeat(latmesh.ravel().tolist(),nolevels)
levs_list = np.repeat(np.expand_dims(zllevels,0),sizemesh,axis=0).ravel()
days_list = [0]* noparts

createParticlesNC('drifters.res.nc',ids=id_list,ines=ines_list ,jnes=jnes_list,start_lats = start_lats_list, start_lons = start_lons_list, lons=lons_list,lats=lats_list,levs=levs_list,days=days_list,start_ds=start_ds_list)

In [3]:
#first = xr.open_dataset('/vortex1/jupyter-kuyeda/Drifters/drifters.res.nc')